# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [74]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine, inspect
import pickle

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
engine = create_engine('sqlite:///DisasterResponse.db')
inspector = inspect(engine)
schemas = inspector.get_schema_names()
for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        print("table_name: %s" % table_name)

schema: main
table_name: Message


In [26]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message', engine, index_col = 'index')
X = df['message'].values
Y = df.iloc[:, 4:].values

In [42]:
df.iloc[:, 4:].columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [69]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize))
    , ('tfidf', TfidfTransformer())
    , ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)) 
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
model = pipeline
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [67]:
df_col = df.iloc[:, 4:].columns.values

In [71]:
for i in range(len(df_col)):
    print(df_col[i], confusion_matrix(y_test[:,0], y_pred[:,0]))

related [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
request [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
offer [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
aid_related [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
medical_help [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
medical_products [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
search_and_rescue [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
security [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
military [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
child_alone [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
water [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
food [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
shelter [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
clothing [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
money [[ 531 1037    4]
 [ 331 4603    5]
 [   2   33    8]]
missing_people [[ 531 1037   

In [72]:
for i in range(len(df_col)):
    print(df_col[i], classification_report(y_test[:,i], y_pred[:,i]))

related              precision    recall  f1-score   support

          0       0.61      0.34      0.44      1572
          1       0.81      0.93      0.87      4939
          2       0.47      0.19      0.27        43

avg / total       0.76      0.78      0.76      6554

request              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5461
          1       0.80      0.40      0.53      1093

avg / total       0.88      0.88      0.87      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6532
          1       0.00      0.00      0.00        22

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.71      0.89      0.79      3829
          1       0.76      0.49      0.59      2725

avg / total       0.73      0.72      0.71      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [90]:
parameters =  {
        'estimator__vect__ngram_range': ((1, 1), (1, 2)),
        'estimator__vect__max_df': (0.5, 0.75, 1.0),
        'estimator__vect__max_features': (None, 5000, 10000),
        'estimator__tfidf__use_idf': (True, False),
        'estimator__clf__n_estimators' : [50, 100, 200],
        'estimator__clf__min_samples_split' : [2, 5, 10],
        'estimator__clf__min_samples_leaf' : [1, 2, 4],
        'estimator__clf__bootstrap' : (True, False)
        }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [93]:
model = cv
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

ValueError: Invalid parameter estimator for estimator Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [92]:
for param in cv.get_params().keys():
    print(param)

cv
error_score
estimator__memory
estimator__steps
estimator__vect
estimator__tfidf
estimator__clf
estimator__vect__analyzer
estimator__vect__binary
estimator__vect__decode_error
estimator__vect__dtype
estimator__vect__encoding
estimator__vect__input
estimator__vect__lowercase
estimator__vect__max_df
estimator__vect__max_features
estimator__vect__min_df
estimator__vect__ngram_range
estimator__vect__preprocessor
estimator__vect__stop_words
estimator__vect__strip_accents
estimator__vect__token_pattern
estimator__vect__tokenizer
estimator__vect__vocabulary
estimator__tfidf__norm
estimator__tfidf__smooth_idf
estimator__tfidf__sublinear_tf
estimator__tfidf__use_idf
estimator__clf__estimator__bootstrap
estimator__clf__estimator__class_weight
estimator__clf__estimator__criterion
estimator__clf__estimator__max_depth
estimator__clf__estimator__max_features
estimator__clf__estimator__max_leaf_nodes
estimator__clf__estimator__min_impurity_decrease
estimator__clf__estimator__min_impurity_split
esti

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# filename = 'classifier.py'
# pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.